In [ ]:
import my_pickle as mp
import my_features as mf

from importlib import reload

In [ ]:
# reload(mp)
reload(mf)

# LOAD DATA FRAMES

In [2]:
# Load User Data
user_df = mp.unjson_it('data_user')
user_df = mp.reinstate_date(user_df,['activeAt', 'available', 'birthday','created','updated'])

In [3]:
# Load Conversation Data
convo_df = mp.unjson_it('data_convo')

In [ ]:
# Import Message DF
message_df = mp.unjson_it('data_message')

# MERGE USER DATA TO CONVERSATION DATA

In [4]:
# add a flag so we can identify rows with missing user data
user_df['flag'] = True

# get labels for new columns
old_names = user_df.columns
new_names = {old:old+'_sender' for old in old_names}

# merge and rename
master_df = convo_df.join(user_df, on='uid_sender') \
        .join(user_df, on='uid_receiver', rsuffix="_receiver") \
        .rename(index=str, columns = new_names)

In [5]:
master_df.head()

,conversation_id,convo_length,len_receiver,len_sender,mid_receiver,mid_sender,response,time_to_respond,timestamp,timestamp_receiver,...,question_ratio_receiver,sentence_count_receiver,sentence_ratio_receiver,smokingOk_receiver,term_receiver,timeframe_receiver,type_receiver,updated_receiver,work_receiver,flag_receiver
100,1Jqbth3nqH0EWNOmyQmW,1,NaN,17.0,None,-Kt7F-lKnoanl54lMnUO,False,NaN,2017-09-03 14:58:30.759,NaT,...,0.0,6.0,0.015789,0.0,NaN,NaN,NaN,2018-01-12 14:10:21.838,Insightly,True
10002,caEwxVJXPTGyR6cCT5K8,1,NaN,9.0,None,-KhP4gs1cj3-MfN7aZkb,False,NaN,2017-04-10 22:58:15.738,NaT,...,0.0,10.0,0.022272,1.0,NaN,NaN,NaN,2018-01-12 10:40:00.228,Shamblott Family Dentistry,True
10003,caZHBg3cEbGhtafu1Ef5,1,NaN,36.0,None,-KnNpug-1BJr75KDw7lH,False,NaN,2017-06-24 06:43:51.751,NaT,...,0.0,1.0,0.005319,1.0,12.0,NaN,1.0,2018-01-12 12:48:19.532,None,True
10008,ccY3lAJp8URACdaEvb0s,1,NaN,24.0,None,-KuB6nmPi4W96oF6J_pc,False,NaN,2017-09-16 19:16:51.128,NaT,...,0.0,0.0,0.000000,1.0,24.0,8.096447e+09,1.0,2018-01-12 16:26:34.803,The Black Cat,True
10009,ccYEEH0Ug7FH2hR293u1,2,26.0,16.0,-KindiRhJWOmeQ0Nt43Q,-KimFk44pXMwVVvWqAVl,True,0.0,2017-04-28 01:53:07.603,2017-04-28 08:21:51.716,...,0.0,5.0,0.020000,0.0,12.0,NaN,1.0,2018-01-12 11:18:39.313,None,True


# DROP CONVERSATIONS WITH MISSING USER DATA

In [ ]:
initial_len = len(master_df)

# add False values
master_df.flag_receiver = master_df.flag_receiver.apply(lambda x: x if x==True else False)

# keep only rows with the flag
master_df = master_df[master_df.flag_receiver]

# see how many conversations we lost
print("{} conversations were removed because they were missing user data".format(initial_len-len(master_df)))

# ADD FEATURES

In [ ]:
# get most of the features
master_df = mf.get_features(master_df)

In [ ]:
# add nlp stuff (should be better integrated)
text_similarity_df = mp.unjson_it('data_text_similarity')
master_df['count_similarity'] = text_similarity_df['count_similarity']
master_df['tfidf_similarity'] = text_similarity_df['tfidf_similarity']

# CONVERT TO X AND y

In [ ]:
keep_X = ['I_count_receiver', 'I_count_sender',  
        'age_dif', 'age_receiver', 'age_sender', 'distance', 
        'exclaim_count_receiver','exclaim_count_sender', 'gender_receiver', 'gender_sender',
        'has_about_receiver', 'has_about_sender', 'has_facebookId_receiver','has_facebookId_sender',
        'has_linkedinId_receiver','has_linkedinId_sender', 'has_picture_receiver','has_picture_sender', 
        'has_room_receiver', 'has_room_sender','inRelationship_receiver', 'inRelationship_sender',
        'isClean_receiver', 'isClean_sender', 'isNight_receiver','isNight_sender', 
        'isStudent_receiver', 'isStudent_sender','len_about_receiver', 'len_about_sender',
        'maxCost_receiver','maxCost_sender','minCost_receiver', 'minCost_sender',
        'numRoommates_receiver','numRoommates_sender',
          'overlap_amenities', 'overlap_hobbies','overlap_neighborhoods','overlap_rent','overlap_roommate',
        'period_count_receiver', 'period_count_sender','petsOk_receiver','petsOk_sender',
        'question_count_receiver', 'question_count_sender',
        'same_city', 'same_clean',
        'same_college', 'same_country', 'same_gender', 'same_metro',
        'same_night', 'same_relate', 'same_smoking', 'same_state',
        'same_student', 'same_term', 'same_type', 'same_work',
        'sentence_count_receiver', 'sentence_count_sender',
        'smokingOk_receiver', 'smokingOk_sender', 'term_receiver','term_sender', 
        'type_receiver', 'type_sender','urgency_receiver', 'urgency_sender']
   
keep_y = ['convo_length','response']
          
drop = ['I_ratio_receiver', 'I_ratio_sender','about_receiver', 'about_sender', 
        'activeAt_receiver', 'activeAt_sender',
        'amenities_receiver', 'amenities_sender','available_receiver', 'available_sender', 
        'birthday_receiver', 'birthday_sender',
        'college_receiver', 'college_sender','created_receiver', 'created_sender',
        'exclaim_ratio_receiver','exclaim_ratio_sender',
        'facebookId_receiver', 'facebookId_sender', 'flag_receiver', 'flag_sender',
        'hobbies_receiver', 'hobbies_sender','hometownCity_receiver', 'hometownCity_sender',
        'hometownCountry_receiver', 'hometownCountry_sender','hometownState_receiver', 'hometownState_sender',
        'len_receiver','len_sender','linkedinId_receiver', 'linkedinId_sender',
        'location_receiver', 'location_sender','metro_receiver', 'metro_sender', 
        'mid_receiver','mid_sender', 'neighborhoods_receiver','neighborhoods_sender',
        'onboarded_receiver', 'onboarded_sender','period_ratio_receiver', 'period_ratio_sender', 
        'picture_receiver', 'picture_sender','question_ratio_receiver', 'question_ratio_sender',
        'sentence_ratio_receiver', 'sentence_ratio_sender','timestamp',
        'uid_receiver', 'uid_sender','updated_receiver', 'updated_sender',
        'work_receiver', 'work_sender']

In [ ]:
X_df = master_df[keep_X]
X_df = X_df.fillna(X_df.mean(axis=0))
y_df = master_df[keep_y]

# PICKLE

In [ ]:
mp.json_it(master_df,'data_master')
mp.json_it(X_df,'data_X')
mp.json_it(y_df,'data_y')